## SST 
- 영화리뷰 추출 12000개 긍부정 라벨
- 0: negative // 1: positive
- huggingface, github 어디서 다운 받아서 사용하든 상관 X
- Navie bayes 구현은 가능한만큼

### 필요한 것
- 긍정, 부정 클래스에서 각 단어가 있는 횟수를 알 수 있는 BoW

In [18]:
# !pip install pyarrow
# !pip install fastparquet
# !pip install scikit-learn

In [3]:
import pandas as pd

df = pd.read_parquet("data/train-00000-of-00001.parquet")
print(df)
print("------------------------------------")
print(df["sentence"][2])

# 감성 분석을 위한 최적화로 각 문서에서 단어가 중복되지 않게 변형
df['sentence'] = df['sentence'].apply(lambda x: ' '.join(set(x.strip().split(' '))))

         idx                                           sentence  label
0          0       hide new secretions from the parental units       0
1          1               contains no wit , only labored gags       0
2          2  that loves its characters and communicates som...      1
3          3  remains utterly satisfied to remain the same t...      0
4          4  on the worst revenge-of-the-nerds clichés the ...      0
...      ...                                                ...    ...
67344  67344                               a delightful comedy       1
67345  67345                   anguish , anger and frustration       0
67346  67346  at achieving the modest , crowd-pleasing goals...      1
67347  67347                                  a patient viewer       1
67348  67348  this new jangle of noise , mayhem and stupidit...      0

[67349 rows x 3 columns]
------------------------------------
that loves its characters and communicates something rather beautiful about human nat

In [4]:
# 필요한 데이터인 sentence와 label만을 추출
all_corpus = df[['sentence', 'label']]

print(all_corpus)

                                                sentence  label
0            new parental secretions from units the hide      0
1                    contains no , only wit gags labored      0
2      communicates that loves human nature rather an...      1
3      throughout satisfied remains same utterly the ...      0
4      worst up could on dredge clichés the filmmaker...      0
...                                                  ...    ...
67344                                comedy delightful a      1
67345                    frustration anguish , anger and      0
67346  crowd-pleasing achieving itself it , sets mode...      1
67347                                   patient viewer a      1
67348  serious stupidity mayhem contender , title . n...      0

[67349 rows x 2 columns]


In [5]:
import nltk
import math
from sklearn.feature_extraction.text import CountVectorizer

def train_naive_bayes(D, C):
    """
    N_doc: 전체 문서의 개수
    all_BoW: 전체 단어의 BoW
    len_all_BoW: 전체 단어의 수

    N_c: 클래스 문서의 개수
    """

    N_doc = len(D)
    all_vect = CountVectorizer(tokenizer=nltk.word_tokenize)
    all_BoW = all_vect.fit(D['sentence']) # 전체 단어에 대해 fit
    len_all_BoW = len(all_BoW.vocabulary_)

    value = {}
    for c in C:
        N_c = len(D[D['label'] == c]) 
        log_prior = math.log(N_c/N_doc)
        print("log_prior:", log_prior)

        bigdoc = D[D['label'] == c]['sentence'].values.astype('U')  # 문장 리스트로 변환
        c_BoW = all_BoW.transform(bigdoc)
        len_c_BoW = c_BoW.nnz
        print("len_c_BoW:", len_c_BoW)

        log_likelihood = {}
        for word, _ in all_BoW.vocabulary_.items():
            word_idx = all_BoW.vocabulary_.get(word)
            word_cnt = c_BoW[:, word_idx].sum()
            log_likelihood[word] = math.log((word_cnt + 1) / (len_c_BoW + len_all_BoW))
        
        value[c] = [log_prior, log_likelihood]
    
    """
    value: 클래스별 log_prior와 log_likelihood
    all_BoW: 전체 단어에 대한 BoW
    C: 분류한 클래스
    """
    return value, all_BoW, C

In [6]:
model = train_naive_bayes(all_corpus, [0, 1]) # 모델 학습

c:\Anaconda3\envs\PTLLM\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


log_prior: -0.8160510276803322
len_c_BoW: 276619
log_prior: -0.5837088126861899
len_c_BoW: 318983


In [7]:
model

({0: [-0.8160510276803322,
   {'new': -7.049155335866165,
    'parental': -10.636634675538373,
    'secretions': -11.196250463473795,
    'from': -5.777930304531061,
    'units': -11.483932535925575,
    'the': -3.480012405251808,
    'hide': -10.017595467132148,
    'contains': -9.638105845427244,
    'no': -5.941362654853094,
    ',': -3.60906667607265,
    'only': -6.479986229980116,
    'wit': -8.488200262371585,
    'gags': -8.039250042323681,
    'labored': -8.999025886137575,
    'communicates': -12.582544824593686,
    'that': -4.452780378799513,
    'loves': -10.184649551795316,
    'human': -8.363037119417578,
    'nature': -8.868972757889377,
    'rather': -7.162009825321399,
    'and': -3.725599068434665,
    'beautiful': -9.74933148053747,
    'about': -5.856311422234938,
    'its': -5.17038448964848,
    'characters': -6.226437163897794,
    'something': -7.029585239672068,
    'throughout': -9.586812551039694,
    'satisfied': -11.196250463473795,
    'remains': -8.97162

In [8]:
import numpy as np

def test_naive_bayes(test_doc, model):
    value, V, C = model

    class_sum = []
    for c in C:
        sum = value[c][0] # log_prior
        words = nltk.word_tokenize(test_doc) # 학습할 때와 동일한 토크나이저 사용
        for word in words:
            if word in V.vocabulary_:
                sum += value[c][1][word] # log_likelihood
        class_sum.append(sum)

    return np.argmax(class_sum), class_sum

In [9]:
c, class_sum = test_naive_bayes("An exciting story... a movie is fantastic.", model)
print(f"classification result: {'negative' if c == 0 else 'positive'}\nnegative: {class_sum[0]} | positive: {class_sum[1]}")

classification result: positive
negative: -51.44199785920988 | positive: -47.17428268288052


In [10]:
df = pd.read_parquet("data/validation-00000-of-00001.parquet")
print(df)
print("------------------------------------")
print(df["sentence"][2])

# 감성 분석을 위한 최적화로 각 문서에서 단어가 중복되지 않게 변형
df['sentence'] = df['sentence'].apply(lambda x: ' '.join(set(x.strip().split(' '))))

     idx                                           sentence  label
0      0    it 's a charming and often affecting journey .       1
1      1                 unflinchingly bleak and desperate       0
2      2  allows us to hope that nolan is poised to emba...      1
3      3  the acting , costumes , music , cinematography...      1
4      4                  it 's slow -- very , very slow .       0
..   ...                                                ...    ...
867  867              has all the depth of a wading pool .       0
868  868              a movie with a real anarchic flair .       1
869  869  a subject like this should inspire reaction in...      0
870  870  ... is an arthritic attempt at directing by ca...      0
871  871  looking aristocratic , luminous yet careworn i...      1

[872 rows x 3 columns]
------------------------------------
allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . 


In [11]:
df['prediction'] = df['sentence'].apply(lambda x: test_naive_bayes(x, model)[0])
print(df)

     idx                                           sentence  label  prediction
0      0     journey affecting it 's charming and a often .      1           1
1      1                  unflinchingly desperate bleak and      0           0
2      2  that major yet poised inventive hope us filmma...      1           1
3      3  acting costumes sound astounding 's , producti...      1           1
4      4                             slow -- 's , it . very      0           0
..   ...                                                ...    ...         ...
867  867               pool has of depth the a wading . all      0           0
868  868                 anarchic flair with movie a real .      1           1
869  869  inspire like ; pianist . reaction audience sho...      0           0
870  870  callie . directing ... is by attempt khouri an...      0           0
871  871  that 's , jane in looking yet exemplary could ...      1           1

[872 rows x 4 columns]


In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 예측된 레이블과 실제 레이블을 추출합니다.
y_pred = df['prediction']
y_true = df['label']

# 정확도, 정밀도, 재현율, F1 점수 계산
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# 결과 출력
print(f'정확도: {accuracy:.4f}')
print(f'정밀도: {precision:.4f}')
print(f'재현율: {recall:.4f}')
print(f'F1 점수: {f1:.4f}')

정확도: 0.8016
정밀도: 0.7877
재현율: 0.8356
F1 점수: 0.8109


In [13]:
df = pd.read_parquet("data/test-00000-of-00001.parquet")
print(df)
print("------------------------------------")
print(df["sentence"][2])

# 감성 분석을 위한 최적화로 각 문서에서 단어가 중복되지 않게 변형
df['sentence'] = df['sentence'].apply(lambda x: ' '.join(set(x.strip().split(' '))))

       idx                                           sentence  label
0        0             uneasy mishmash of styles and genres .     -1
1        1  this film 's relationship to actual tension is...     -1
2        2  by the end of no such thing the audience , lik...     -1
3        3  director rob marshall went out gunning to make...     -1
4        4  lathan and diggs have considerable personal ch...     -1
...    ...                                                ...    ...
1816  1816  it risks seeming slow and pretentious , becaus...     -1
1817  1817  take care of my cat offers a refreshingly diff...     -1
1818  1818  davis has filled out his cast with appealing f...     -1
1819  1819  it represents better-than-average movie-making...     -1
1820  1820  dazzling and sugar-sweet , a blast of shallow ...     -1

[1821 rows x 3 columns]
------------------------------------
by the end of no such thing the audience , like beatrice , has a watchful affection for the monster .


In [33]:
df['label'] = df['sentence'].apply(lambda x: test_naive_bayes(x, model)[0])
print(df)

       idx                                           sentence  label
0        0             . mishmash of genres styles uneasy and      0
1        1  -- : a snow if . 's this spray in same christm...      0
2        2  no for beatrice audience thing , a end watchfu...      0
3        3  out marshall a . director make rob went gunnin...      1
4        4  old considerable their , rapport story . perso...      1
...    ...                                                ...    ...
1816  1816  risks promise because gamble slow pretentious ...      0
1817  1817  care cinema a take offers asian refreshingly m...      1
1818  1818  out faces appealing cast davis has fresh . fil...      1
1819  1819  n't a that movie-making , audience . demand di...      0
1820  1820  a chorus deliver . blast sugar-sweet dazzling ...      1

[1821 rows x 3 columns]
